In [8]:
#先把套路的东西给搞定了
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
from operator import itemgetter


class Chimerge:
    '''
    <卡方分箱😍😍>\n
    输入数据，dataframe形式\n
    args: \n
    1.数据 dataframe\n
    格式是 X1,X2...Y<还有X0,代码...>\n
    2.最小箱数 int\n
    3.最大卡方 int\n
    4.缺失值的值 ：）
    '''
    def __init__(self,
                data,
                minbin = 8,
                maxchi = 3,
                misval = -9999):
        self.maxchi = maxchi
        self.minbin = minbin
        self.misval = misval
        data['good'] = data['fund_fix']
        data['bad'] = data['fund_fix'].replace({0:1,1:0})
        data['chi'] = 0
        self.data = data.drop('fund_fix',axis = 1)
        self.label = self.data.columns
        self.scale = self.data.shape[0]
        #好坏各为一列
        self.datar = self.data.values
        self.good = sum(self.data.loc[:,'good'])
        self.bad = sum(self.data.loc[:,'bad'])
        self.misscount = []
        self.missratio = []
        self.chi = []
        self.output = pd.DataFrame()
    
    
    def _chivalue(self,i):
        '''
        输入序号，输出chi的值\n
        args:\n
        1.箱1序号\n
        '''
        total = self.good + self.bad
        gdrt = self.good/total
        bdrt = self.bad/total
        gd1 = self.datar[i][1]
        bd1 = self.datar[i][2]
        gd0 = self.datar[i-1][1]
        bd0 = self.datar[i-1][2]
        Egd1 = (gd1+bd1)*gdrt
        Egd0 = (gd0+bd0)*gdrt
        Ebd1 = (gd1+bd1)*bdrt
        Ebd0 = (gd0+bd0)*bdrt
        return self._sq(bd1-Ebd1)/Ebd1 + self._sq(bd0-Ebd0)/Ebd0 + self._sq(gd1-Egd1)/Egd1 + self._sq(gd0-Egd0)/Egd0

    def _rchivalue(self,i):
        gd1 = self.datar[i][1]
        bd1 = self.datar[i][2]
        gd0 = self.datar[i-1][1]
        bd0 = self.datar[i-1][2]
        total = gd1 + bd1 + gd0 + bd0
        Ex1gd = (gd1+gd0)*(gd1+bd1)/(total)
        Ex1bd = (bd1+bd0)*(gd1+bd1)/(total)
        Ex0gd = (gd1+gd0)*(gd0+bd0)/(total)
        Ex0bd = (bd1+bd0)*(gd0+bd0)/(total)
        Chivalue = (Ex1gd - gd1)**2/Ex1gd + (Ex1bd - bd1)**2/Ex1bd + (Ex0gd - gd0)**2/Ex0gd + (Ex0bd - bd0)**2/Ex0bd
        return Chivalue


    def _merge2bin(self,i):
        '''
        输入序号，然后合并，无输出\n
        args: \n
        1.箱1序号
        '''
        self.datar[i][1] = self.datar[i][1] + self.datar[i-1][1]
        self.datar[i][2] = self.datar[i][2] + self.datar[i-1][2]
        self.datar = np.delete(self.datar,i-1,axis = 0)
        #合并
        if i != 2:
            self.datar[i-1][3] = self._rchivalue(i-1)
        if i < self.datar.shape[0]:
            self.datar[i][3] = self._rchivalue(i)
        #更新卡方值
    
    def _cal_all_chi(self):
        '''
        算所有的卡方值，然后填进矩阵
        '''
        self.datar[0,3] = 9999
        for i in range(1,self.datar.shape[0]):
            self.datar[i,3] = self._rchivalue(i)

    def _findmin(self):
        '''
        返回最小卡方值的序号
        '''
        self.chi = self.datar[:,3]
        return np.argmin(self.chi)
        
    def suan(self):
        '''
        开算！\n
        结果输出到.output里
        '''
        if self.datar.shape[0] <= self.minbin:
            self.check_datar()
        else:
            self.eqfrebin()

        self._cal_all_chi()
        if self.datar.shape[0] > self.minbin:
            while True:
                index = self._findmin()
                if self.datar.shape[0] <= self.minbin:
                    break
                if min(self.chi) >= self.maxchi:
                    break 
                self._merge2bin(index)
        #把缺失值整进去
        val = self.datar[:,0].tolist()
        val.append('缺失')
        good = self.datar[:,1].tolist()
        good.append(self.misscount[1])
        bad = self.datar[:,2].tolist()
        bad.append(self.misscount[2])
        self.output = pd.DataFrame({'value':val,'good':good,'bad':bad})

    def sort(self,data,index):
        '''
        只按照指定列的数据，输出排序后数列\n
        1.数据\n
        2.指定列序号
        '''
        a1 = data.tolist()
        a1 = sorted(a1,key=itemgetter(index))
        return np.array(a1,dtype = 'O')
    
    def _cal_misscount(self):
        if np.where(self.datar[:,0] == self.misval)[0].size != 0:
            self.misscount = self.datar[np.where(self.datar == self.misval)[0]][0][:3].tolist()
            self.datar = np.delete(self.datar,np.where(self.datar == self.misval)[0],axis = 0)
        else:
            self.misscount = [-9999,0,0]

    def initailize(self):
        '''
        初始化，包含切出缺失值，小到大排序,和准备一个单值表
        ''' 
        self._cal_misscount()
        self.datar = self.sort(self.datar,0)
        self.ori = self.datar[:,:3]
        self.ori = np.vstack([self.misscount,self.ori])
        #这里先存了一个表，是每个值单列出来的，用于建立WOE与每个值的关系表

    def cal_missing_ratio(self):
        '''
        计算各个变量的缺省值，输出一个表
        '''
        for i in range(0,self.data.shape[1]):
            length = self.data.shape[0]
            count = self.data.loc[self.data.iloc[:,i]==self.misval].shape[0]
            self.missratio.append(count/length)

    def _kickvalue(self, value = 0.9):
        '''
        输入一个上限，默认0.9,返回被T的变量名\n
        value = 上限
        '''
        index = np.where(np.array(self.missratio) > value)[0]
        keys = self.label[index]
        self.data.drop(keys.tolist(),axis = 1,inplace = True)
        self.label = self.data.columns
        return keys
    
    def split(self,index = None,i = None):
        '''
        切出表里第i个数据，化成该有的形式，填到datar里\n
        1.输入变量的名称，默认 None\n
        2.输入变量位置，默认 None
        '''
        if i is None:
            self.datar = self.data[[index,'hao','huai','chi']].values
            self.good = np.sum(self.datar[:,1])
            self.bad = np.sum(self.datar[:,2])
            #懒得优化了
        elif type(i) == int:
            self._drop_dup(i)
            self.i = i
        else:
            print("😅😅")

    def _drop_dup(self,i):
        '''
        去重，且把相同的项相“加”
        '''
        data1 = self.data.iloc[:,[i,self.data.shape[1]-3,self.data.shape[1]-2]].values
        vallist = self.data.iloc[:,i].drop_duplicates().values
        #vallist = np.unique(data1[:,0])
        data_i = [[] for i in range(len(vallist))]
        for j in range(0,len(vallist)):
            cut_i = data1[np.where(data1[:,0] == vallist[j])[0]]
            data_i[j].append(vallist[j])
            data_i[j].append(sum(cut_i[:,1]))
            data_i[j].append(sum(cut_i[:,2]))
            data_i[j].append(0)
        self.datar = np.array(data_i,dtype = 'O')

    def check_datar(self):
        '''
        检查分箱后的数据
        1.某些箱体数据过少,且某个变量含0，故需合并
        '''
        times = 0
        while np.where(self.datar[:,[1,2]] == 0)[0].size != 0:
            '''存在某箱，含数很少'''
            a = np.where(self.datar[:,[1,2]] == 0)
            index0 = a[0][0]
            index1 = a[1][0]
            '''翻出第一个 "错误的"箱'''
            if index1 == 1:
                if self.datar[index0,2] >= 100:
                    print(f'{self.i}这个变量有点好')
            else:
                if self.datar[index0,1] >= 100:
                    print(f'{self.i}这个变量有点好')
            if index0 != 0:
                self._merge2bin(index0)
                times += 1
            else:
                print(f'{self.i}第一位不对劲')
        if times >= 20:
            print(f'注意一下{self.i}😁😁')

    def eqfrebin(self,binnum = 20,way = True):
        '''
        🤡🤡🤡
        '''
        binsval = int((self.scale - self.misscount[1] - self.misscount[2])/binnum)
        size = self.datar[:,0].size
        #这个循环重复次数很短，仅仅20次，可以说是微不足道了
        if way:
            outcome = [[]]
            start = size
            end = size - 1
            i = 0
            while (end >= 0) & (binnum > 0):
                all = sum(self.datar[end:start,[1,2]])
                if sum(all) < binsval:
                    #两次求和才能求出来
                    end -= 1
                else:
                    binnum = binnum - sum(self.datar[end,[1,2]])//binsval - 1
                    outcome[i].append(self.datar[start-1,0])
                    outcome[i].append(all[0])
                    outcome[i].append(all[1])
                    outcome[i].append(0)
                    outcome.append([])
                    i += 1
                    start = end
                    end -= 1
        outcome.pop()
        self.datar = np.array(outcome)

import re
def encoder(df,misval = -9999):
    '''
    将输入的一列有序的<string>转换为数字编码输出，即按照分类[0,1,2,3...]这样编码\n
    输入默认为dataframe\n
    一定要数据类似，比如1-2与3-4，字符串和数据连在一起就寄了，
    '''
    a = df.astype('str')
    a1 = a.drop_duplicates().values.tolist()
    #喜闻乐见去缺失
    a1.remove(str(misval))
    result = []
    for i in a1:
        b = re.split('\+|-|<|>',i)
        if (b[0].isdigit()) | (b[1].isdigit()):
            if len(b[0]) != 0:
                result.append(int(float(b[0])))
            else:
                result.append(int(float(b[1])))
        else:
            result.append(b[0])
    #处理烦人的区间模式
    a1,a2 = np.array(a1),np.array(result)
    index = np.argsort(a2)
    vallist = a1[index]
    num = [i for i in range(len(a1))]
    dic = dict(zip(vallist,num))
    print(dic)
    df.replace(dic,inplace = True)
    return df




class WOE:
    '''
    输入分箱后的多维数组：
    把WOE值输出为新的一列 并列表
    '''
    def __init__(self,data):
        self.data = data
        self.gd = self.data.loc[:,'good'].values
        self.bd = self.data.loc[:,'bad'].values
        self.bins = self.data.loc[:,'value'].values
        self.WOEval = []
        self.IV = 0
    def _cal_WOE(self,index):
        '''
        算一个WOE值\n
        1.数据序号
        '''
        gd = self.gd[index]/sum(self.gd)
        bd = self.bd[index]/sum(self.bd)
        if (gd != 0 )&(bd != 0):
            Woe = math.log(gd/bd)
        elif gd == 0:
            Woe = -999999
        else:
            Woe = 999999
        IVi = Woe*(gd-bd)
        return [Woe,IVi]
    
    def initialize(self):    
        '''
        计算所有WOE值，并填入表
        并且计算IV值
        '''
        for i in range(0, self.data.shape[0]):
            self.WOEval.append(self._cal_WOE(i)[0])
            self.IV += self._cal_WOE(i)[1]
        self.data['WOE'] = self.WOEval

    def plot(self):
        '''
        根据WOE值画幅图
        '''
        pass

from sklearn.linear_model import LogisticRegression
class logisticregression():
    '''
    有趣的回归<单纯引库而已>
    '''
    #默认结构是 0变量值 1好值 2坏值 3WOE值
    def __init__(self,data):
        self.data = data

    def single_regression(self):
        #先把数据还原为最初的形态
        x_train = []
        y_train = []
        
        for i in range(0, self.data.shape[0]):
            x_train.append([self.data[i,0] for j in range(0,sum(self.data[i,[1,2]]))])
            y_train.append([1 for j in range(0,self.data[i,1])])
            y_train.append([0 for j in range(0,self.data[i,2])])
        
        logit = LogisticRegression(penalty='l2',random_state=2020)
        logit.fit(x_train,y_train)

    def multi_regression(self):
        pass

    def validate_regression(self):
        '''
        通过算测试集的
        '''

    def _build(self,table1,table2,table3):
        '''
        旨在构造单值对应woe的表\n
        默认都是矩阵，且为0值1好2坏3WOE（可能没有）\n
        表一：区间对应WOE表 \n
        表二：全取值 \n
        表三：需要匹配上WOE
        ''' 
        df1 = pd.DataFrame(table1,columns = ['val','gd','bd','WOE'])
        df2 = pd.DataFrame(table2,columns = ['val','gd','bd'])
        df3 = pd.DataFrame(table3,columns = ['val','gd','bd'])
        
        df1 = pd.merge(df2['val'],df1,how = 'left', on = 'val')
        for i in range(df1.shape[0]):
            if not df1['WOE',df1.shape[0] - i - 1].isnull():
                a = df1['WOE',df1.shape[0] - i - 1]
            else:
                df1['WOE',df1.shape[0] - i - 1] = a
        #这样df1就成了WOE和单值之间的对应了
        
        df3 = pd.merge(df3['val'],df1.loc[:,['val','WOE']], how = 'left', on ='val')
        #用测试集分箱，用全集比对woe表，